In [3]:
import pandas as pd, os, re, string
import numpy as np
import spacy
nlp = spacy.load('en_core_web_md')


In [7]:
from nltk.tokenize import sent_tokenize
#import nltk; nltk.download('stopwords')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('punkt')
#from nltk.stem import PorterStemmer
#from nltk.corpus import wordnet
#from bs4 import BeautifulSoup
#nltk.download('words')
from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords

In [4]:
comp=pd.read_excel(r'C:\Saurabh\Publication\Chunking\Chunk_data.xlsx')
print(len(comp))

2

In [5]:
print(comp.dtypes)


Text    object
dtype: object


In [94]:
print(comp['Text'].astype(str))

0    Ruth Bader Ginsburg was an associate justice o...
1    I opened a savings account at Wells Fargo in D...
Name: Text, dtype: object


In [8]:
comp['sentence'] = comp['Text'].apply(lambda x: sent_tokenize(x))
len(comp['sentence'][0])

2

In [9]:
#Keeping only text from the consumer complain
def replace_value(s):
    s = re.sub(r' [X]+ ','',s)
    s = re.sub(r'[XX]+','',s)
    s = re.sub('[^a-zA-Z]', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s 
 
#comp['sentence_clean']=comp['sentence'].apply(lambda x: [replace_value(y).strip().lower() for y in x])
comp['sentence_clean'] = comp['sentence']


In [10]:
print(comp['sentence_clean'])

0    [Ruth Bader Ginsburg was an associate justice ...
1    [I opened a savings account at Wells Fargo in ...
Name: sentence_clean, dtype: object


In [100]:

comp_subset = pd.DataFrame(comp['sentence_clean'][0], columns=['sentence_clean'])
comp_subset

,sentence_clean
0,Ruth Bader Ginsburg was an associate justice o...
1,The second of only four women to ever serve on...


In [101]:
comp_subset = comp_subset[comp_subset.index==0]

In [102]:
def onegram(text):
    doc = nlp(text)
    result = []
    print("{0:20} {1:20} {2:8} {3:8} {4:8} {5:8} {6:8} {7:8}".format("text", "lemma_", "pos_", "tag_", "dep_",
            "shape_", "is_alpha", "is_stop"))
    for token in doc:
        print("{0:20} {1:20} {2:8} {3:8} {4:8} {5:8} {6:8} {7:8}".format(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop))
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            #token.shape_, token.is_alpha, token.is_stop)
        result.append((token.text, token.tag_))
    return result

In [103]:
comp_subset.head(1)['sentence_clean'].apply(onegram)

text                 lemma_               pos_     tag_     dep_     shape_   is_alpha is_stop 
Ruth                 Ruth                 PROPN    NNP      compound Xxxx            1        0
Bader                Bader                PROPN    NNP      compound Xxxxx           1        0
Ginsburg             Ginsburg             PROPN    NNP      nsubj    Xxxxx           1        0
was                  be                   AUX      VBD      ROOT     xxx             1        1
an                   an                   DET      DT       det      xx              1        1
associate            associate            ADJ      JJ       amod     xxxx            1        0
justice              justice              NOUN     NN       attr     xxxx            1        0
of                   of                   ADP      IN       prep     xx              1        1
the                  the                  DET      DT       det      xxx             1        1
U.S.                 U.S.               

0    [(Ruth, NNP), (Bader, NNP), (Ginsburg, NNP), (...
Name: sentence_clean, dtype: object

In [104]:
def extractNP(text):
    doc = nlp(text)
    result = []
    for np in doc.noun_chunks:
        result.append(np.text)
    print(result)
    return result

In [105]:
comp_subset['sentence_clean'].apply(extractNP)

['Ruth Bader Ginsburg', 'an associate justice', 'the U.S. Supreme Court', 'nearly three decades']


0    [Ruth Bader Ginsburg, an associate justice, th...
Name: sentence_clean, dtype: object

In [106]:
from spacy import displacy
def getdependency(text):
    doc = nlp(text)
    print("{0:20} {1:20} {2:20} {3:20} {4:30}".format("text", "dep_", "head", "head_pos_", "children"))
    for token in doc:
        print("{0:20} {1:20} {2:20} {3:20} {4:30}".format(token.text, token.dep_, token.head.text, token.head.pos_, 
                                                 " ".join([str(child) for child in token.children])))
    return

def getdisplay(text):
    doc = nlp(text)
    displacy.serve(doc, style="dep")
    return

In [107]:
comp_subset['sentence_clean'].apply(getdependency)

text                 dep_                 head                 head_pos_            children                      
Ruth                 compound             Ginsburg             PROPN                                              
Bader                compound             Ginsburg             PROPN                                              
Ginsburg             nsubj                was                  AUX                  Ruth Bader                    
was                  ROOT                 was                  AUX                  Ginsburg justice .            
an                   det                  justice              NOUN                                               
associate            amod                 justice              NOUN                                               
justice              attr                 was                  AUX                  an associate of for           
of                   prep                 justice              NOUN             

0    None
Name: sentence_clean, dtype: object

In [108]:
comp_subset['sentence_clean'].apply(getdisplay)

C:\Users\saurabh.t.singh\anaconda3\envs\bert36\lib\site-packages\spacy\displacy\__init__.py:94: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


0    None
Name: sentence_clean, dtype: object

In [111]:
def getEntity(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
    return

In [112]:
comp_subset['sentence_clean'].apply(getEntity)

Ruth Bader Ginsburg 0 19 PERSON
the U.S. Supreme Court 48 70 ORG
nearly three decades 75 95 DATE


0    None
Name: sentence_clean, dtype: object

In [109]:
def twogram(text):
    doc = nlp(text)
    result = []
    prev_token = ""
    prev_tag_ = ""
    for token in doc:
        if prev_token != "":
            twogram = prev_token + " " + token.text
            tag_  = (prev_tag_, token.tag_)
            result.append((twogram, tag_))
        prev_token = str(token.text)
        prev_tag_ = token.tag_
    return result

In [130]:
comp_subset.head(1)['sentence_clean'].apply(twogram)

0    [(refuses to, (VBZ, TO)), (to accurately, (TO,...
Name: sentence_clean, dtype: object

In [131]:
def threegram(text):
    doc = nlp(text)
    result = []
    prev_token = ""
    prev_tag_ = ""
    prev_token2 = ""
    prev_tag_2 = ""

    for token in doc:
        if prev_token2 != "":
            threegram = prev_token2 + " " + prev_token + " " + token.text
            tag_  = (prev_tag_2, prev_tag_, token.tag_)
            result.append((threegram, tag_))
        prev_token2 = prev_token
        prev_tag_2 = prev_tag_
        prev_token = str(token.text)
        prev_tag_ = token.tag_
    return result

In [132]:
comp_subset.head(1)['sentence_clean'].apply(threegram)

0    [(refuses to accurately, (VBZ, TO, RB)), (to a...
Name: sentence_clean, dtype: object

Autonomous cars
insurance liability
manufacturers
